In [1]:
import socket
import threading
import numpy as np
import struct
import matplotlib.pyplot as plt
import time
import random

smpnum = 256
pout = 4
dtrLen = smpnum*3000
dtsLen = smpnum*pout + 3
rcvLast = hrtLast = time.time()
reCon = 30  # reconnect time
rdata = np.arange(0, dtrLen, 1)
uprdata = np.zeros(dtrLen)
nloop = range(0,dtrLen)
strrdata = ''.join([str(x) for x in rdata])
print('rdata: ', len(rdata), ',', rdata[:10])
sdata = np.arange(0, dtsLen*0.1, 0.1)*1.1+1
sdata[0] = smpnum*pout
sdata[1] = smpnum
sdata[2] = pout
upsdata = sdata
print('sdata: ', len(sdata), ',', sdata[:10])

rdata:  768000 , [0 1 2 3 4 5 6 7 8 9]
sdata:  1027 , [1024.    256.      4.      1.33    1.44    1.55    1.66    1.77    1.88
    1.99]


# 客户端

In [2]:
def tcpclient():
    print("bytes begain")
    count = 0    
    rcved = 0
    np.set_printoptions(precision=3, suppress=True)
    while count<999:
        count += 1
        recv_data = sd.recv(dtrLen)
        rcv_len = len(recv_data)
        if rcv_len<12:
            time.sleep(0.01)
            continue
        elif rcv_len==12:
            raw_leng, rows, cols = struct.unpack('3i', recv_data)
            print(count, " : ", rcv_len, raw_leng, rows, cols)            
            time.sleep(0.01)
            continue
        waitbytes = recv_data   
        xw = [10, -41, -119, 65]
        x=struct.pack('4b', *xw)
        pfloat=  struct.unpack('<f', x)[0]
        print(pfloat)
        ba = bytearray()
        ba.append(waitbytes[0])
        ba.append(waitbytes[1])
        ba.append(waitbytes[1])
        ba.append(waitbytes[0])
        pfloat=  struct.unpack('!f', ba)[0]
        print(pfloat)

        bnum = int(rcv_len/4);
        pfloat=struct.unpack('!%df'%bnum,waitbytes)[0:bnum]
        print(pfloat[0:10])

        dec_data = struct.unpack('!%df'%bnum, waitbytes)[0:bnum]#小端
        np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
        np_data = np.array(dec_data)
        print(count, " : ", rcv_len, dec_data[12:18], np_data[42:48])
        unpackdata = struct.unpack('4s', waitbytes[0:4])
        unpackstr  = struct.unpack('4i', recv_data[0:16])
        print(unpackdata, unpackstr)
        #sd.sendall(sdata)
        time.sleep(0.01)
    sd.close()    
    print('finished!')
        
sd=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
sd.settimeout(20)
print('尝试连接服务器.....')
sd.connect(('localhost', 6310))  # receive data
print ("已连接到服务器—> \n")
tc=threading.Thread(target=tcpclient,args=())
tc.start()

尝试连接服务器.....
已连接到服务器—> 

bytes begain
17.229999542236328
4.240934513983485e-39
(4.3048000927935526e-39, 3.587324068671532e-43, 4.203895392974451e-42, 0.9573184847831726, 0.7667520642280579, 0.3333474099636078, 0.5809547901153564, 0.9821335077285767, 0.7167133092880249, 0.11398933082818985)
1  :  5840 (0.911037802696228, 0.6444405913352966, 0.8864526748657227, 0.6642016768455505, 0.16794860363006592, 0.9742698669433594) [ 0.293  0.518  0.669  0.350  0.720  0.329]
(b'\x00.\xe0\x08',) (148909568, 65536, -1207238656, -753765057)
17.229999542236328
0.8725470900535583
(0.8732503056526184, 0.8807920217514038, 0.9766004681587219, 0.4798605144023895, 0.1024148017168045, 0.5001866221427917, 0.30695971846580505, 0.22006185352802277, 0.8751440644264221, 0.5988189578056335)
2  :  195640 (0.6203706860542297, 0.13448286056518555, 0.22067609429359436, 0.4085958003997803, 0.1401854306459427, 0.35414448380470276) [ 0.362  0.752  0.734  0.890  0.124  0.159]
(b'?_\x8dU',) (1435328319, -1770299073, 2097314

In [6]:
recv_data

NameError: name 'recv_data' is not defined